# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [31]:
extraction_templates = {
    "invoice":         ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":           ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_transcript": ('../analyzer_templates/call_transcript.json', '../data/callCenterRecording.mp3'),
    "marketing_video": ('../analyzer_templates/marketing_video.json', '../data/video.mp4'              )
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [32]:
import uuid

ANALYZER_TEMPLATE = "invoice"
ANALYZER_ID = "field-extraction-sample-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction",
)

## Create Analyzer from the Template

In [34]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer field-extraction-sample-7e48bcbc-c551-4e17-ba09-cea051ba1557 create request accepted.
INFO:python.content_understanding_client:Request 9a5b5c93-916c-4206-9afe-6d4870f19bf9 in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.27 seconds.


{
  "id": "9a5b5c93-916c-4206-9afe-6d4870f19bf9",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-7e48bcbc-c551-4e17-ba09-cea051ba1557",
    "description": "Sample invoice analyzer",
    "createdAt": "2024-12-10T23:39:42Z",
    "lastModifiedAt": "2024-12-10T23:39:44Z",
    "config": {
      "returnDetails": false,
      "enableOcr": true,
      "enableLayout": true,
      "enableBarcode": false,
      "enableFormula": false
    },
    "fieldSchema": {
      "fields": {
        "VendorName": {
          "type": "string",
          "method": "extract",
          "description": "Vendor issuing the invoice"
        },
        "Items": {
          "type": "array",
          "method": "extract",
          "items": {
            "type": "object",
            "properties": {
              "Description": {
                "type": "string",
                "method": "extract",
                "description": "Description of the item"
              },
            

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [35]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzing file ../data/invoice.pdf with analyzer: field-extraction-sample-7e48bcbc-c551-4e17-ba09-cea051ba1557
INFO:python.content_understanding_client:Request a0a20152-53a2-4608-8567-4be90dc37e39 in progress ...
INFO:python.content_understanding_client:Request a0a20152-53a2-4608-8567-4be90dc37e39 in progress ...
INFO:python.content_understanding_client:Request a0a20152-53a2-4608-8567-4be90dc37e39 in progress ...
INFO:python.content_understanding_client:Request result is ready after 6.62 seconds.


{
  "id": "a0a20152-53a2-4608-8567-4be90dc37e39",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-7e48bcbc-c551-4e17-ba09-cea051ba1557",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2024-12-10T23:39:45Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "CONTOSO LTD.\n\n\n# INVOICE\n\nContoso Headquarters\n123 456th St\nNew York, NY, 10001\n\nINVOICE: INV-100\n\nINVOICE DATE: 11/15/2019\n\nDUE DATE: 12/15/2019\n\nCUSTOMER NAME: MICROSOFT CORPORATION\n\nSERVICE PERIOD: 10/14/2019 - 11/14/2019\n\nCUSTOMER ID: CID-12345\n\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\n\nBILL TO:\n\nMicrosoft Finance\n\n123 Bill St,\n\nRedmond WA, 98052\n\nSHIP TO:\n\nMicrosoft Delivery\n\n123 Ship St,\n\nRedmond WA, 98052\n\nSERVICE ADDRESS:\nMicrosoft Services\n123 Service St,\nRedmond WA, 98052\n\n\n<table>\n<tr>\n<th>SALESPERSON</th>\n<th>P.O. NUMBER</th>\n<th>REQUISITIONER</th>\n<th>SHIPPED VIA</th>\n<th>F.O.B. POINT</th>\n<th>TERMS</

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [36]:
client.delete_analyzer(ANALYZER_ID)

INFO:python.content_understanding_client:Analyzer field-extraction-sample-7e48bcbc-c551-4e17-ba09-cea051ba1557 deleted.


<Response [204]>